In [ ]:
import pandas as pd
import numpy as np
from sklearn import svm, metrics
from sklearn.model_selection import train_test_split

In [ ]:
Frogs_dataset = pd.read_csv('E:\\Data Mining with Python\\Dataset\\Frogs_MFCCs.csv')


In [24]:
# Frogs_dataset.head()

# Frogs_dataset.info()
# Frogs_dataset.isnull().sum()
# Frogs_dataset.columns
# # np.shape(Frogs_dataset)


# Frogs_dataset.Family.value_counts()
# Frogs_dataset.Genus.value_counts()
# Frogs_dataset.Species.value_counts()

Leptodactylidae    4420
Hylidae            2165
Dendrobatidae       542
Bufonidae            68
Name: Family, dtype: int64

In [ ]:
# import seaborn as sns
# import matplotlib.pyplot as plt
# sns.countplot(y="Family", data=Frogs_dataset , hue="Family")
# plt.show()

# sns.countplot(y='Genus', data=Frogs_dataset)
# plt.show()

# sns.countplot(y='Species', data=Frogs_dataset)
# plt.show()

In [27]:
Frogs_dataset.shape

(7195, 26)

In [17]:
X = Frogs_dataset.iloc[:,:-4]
# y = Frogs_dataset['Family'].values
y = Frogs_dataset.iloc[: , -4:-1]

In [ ]:
# from sklearn.preprocessing import StandardScaler

# scaler_X = StandardScaler()
# X = scaler_X.fit_transform(X)
# X

In [22]:
from sklearn.preprocessing import LabelEncoder

labelencoder_y1 = LabelEncoder()
labelencoder_y2 = LabelEncoder()
labelencoder_y3 = LabelEncoder()

labelencoder_y1 = labelencoder_y1.fit(y['Family'])
labelencoder_y2 = labelencoder_y2.fit(y['Genus'])
labelencoder_y3 = labelencoder_y3.fit(y['Species'])

y['Family'] = labelencoder_y1.transform(y['Family'])
y['Genus'] = labelencoder_y2.transform(y['Genus'])
y['Species'] = labelencoder_y3.transform(y['Species'])

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [36]:
# ii:

# support vector classifier (SVC)
# SVC and NuSVC implement the “one-versus-one” approach for multi-class classification.
# rbf => Gaussian Kernel

# decision_function_shape = "ovr"
# decision_function_shape='ovo'

# Proper choice of C and gamma is critical to the SVM’s performance. One is advised to use sklearn.model_selection.GridSearchCV with C and gamma spaced exponentially far apart to choose good values.


from sklearn.model_selection import cross_val_score

model = svm.SVC(kernel= 'rbf')
scores_1 = cross_val_score(model, X=X_train , y=y_train.iloc[:,0] , cv=10 )
scores_2 = cross_val_score(model, X=X_train , y=y_train.iloc[:,1] , cv=10 )
scores_3 = cross_val_score(model, X=X_train , y=y_train.iloc[:,2] , cv=10 )


print('mean of Scores for Class 1:',scores_1.mean())
print('mean of Scores for Class 2:',scores_2.mean())
print('mean of Scores for Class 3:',scores_3.mean())


mean of Scores for Class 1: 0.973789406418631
mean of Scores for Class 2: 0.9741846539808765
mean of Scores for Class 3: 0.9777596326801097


In [62]:
from sklearn.model_selection import GridSearchCV

model = svm.SVC()
parameters = [
    {'kernel': ['rbf'], 'gamma': [.1,.2,.3,.4,.5,.6,.7],'C': [1, 10, 100, 1000] ,'decision_function_shape': ('ovr', 'ovo')},
    {'kernel': ['rbf'], 'C': [1, 10, 100, 1000],'decision_function_shape' : ('ovr', 'ovo')}]


svm_clf = GridSearchCV(estimator=model,
                     param_grid=parameters,
                     scoring='accuracy',
                     cv=10,
                     n_jobs=-1)

svm_clf.fit(X_train, y_train.iloc[:,0])
print('Best parameters for class 1(Family):',svm_clf.best_params_)
print('Best Scores for class 1(Family):',svm_clf.best_score_)

svm_clf.fit(X_train, y_train.iloc[:,1])
print('Best parameters for class 2(Genus):',svm_clf.best_params_)
print('Best Scores for class 2(Genus):',svm_clf.best_score_)

svm_clf.fit(X_train, y_train.iloc[:,2])
print('Best parameters for class 3(Species):',svm_clf.best_params_)
print('Best Scores for class 3(Species):',svm_clf.best_score_)


# expected = y_test.iloc[:,2]
# predicted = svm_clf.predict(X_test)
# print(metrics.classification_report(expected, predicted))

# وقتی حساسیت زیادی میخواهیم ایجاد کنیم
#  recall را زیاد میکنیم
# Recall(R) = TP / (TP+FN)  => FN~0 => Recall = TP/TP = 1
# Precision(P) = TP/(TP+FP)
# f1-score or f-measure = (2*P*R) / (P+R)  => both R and P are involved in the measure
# f-measure keeps the balance between R and P

# if P is 100% we don't have any false prediction


Best parameters for class 1(Family): {'C': 100, 'decision_function_shape': 'ovr', 'gamma': 0.6, 'kernel': 'rbf'}
Best Scores for class 1(Family): 0.990864337782827
Best parameters for class 2(Genus): {'C': 100, 'decision_function_shape': 'ovr', 'gamma': 0.7, 'kernel': 'rbf'}
Best Scores for class 2(Genus): 0.9910631449398846
Best parameters for class 3(Species): {'C': 1000, 'decision_function_shape': 'ovr', 'kernel': 'rbf'}
Best Scores for class 3(Species): 0.9892762472782355


In [38]:
# Classification for Label 'Family'
from sklearn.metrics import accuracy_score,hamming_loss

# best parameters for class 1: Family => {'C': 100, 'decision_function_shape': 'ovr', 'gamma': 0.6, 'kernel': 'rbf'}
svm_clf = svm.SVC(kernel='rbf', gamma=0.6, C=100, decision_function_shape='ovr')
svm_clf = svm_clf.fit(X_train,y_train.iloc[:,0])
predicted = svm_clf.predict(X_test)
expected = y_test.iloc[:,0]

hamming_loss_1 = hamming_loss(y_test.iloc[:,0],predicted)
exact_match_1 = accuracy_score(y_test.iloc[:,0],predicted)

print("Hamming loss for Class 1 (Family):",hamming_loss_1)
print("Exact Match for Class 1 (Family):",exact_match_1)

print('Classification Report:')
print(metrics.classification_report(expected, predicted))
print('Confusion Matrix:')
print(metrics.confusion_matrix(expected, predicted))


Hamming loss for Class 1 (Family): 0.006947660954145438
Exact Match for Class 1 (Family): 0.9930523390458545
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.95      0.97        20
           1       1.00      0.99      1.00       155
           2       0.99      0.99      0.99       657
           3       0.99      0.99      0.99      1327

    accuracy                           0.99      2159
   macro avg       1.00      0.98      0.99      2159
weighted avg       0.99      0.99      0.99      2159

Confusion Matrix:
[[  19    0    0    1]
 [   0  154    1    0]
 [   0    0  651    6]
 [   0    0    7 1320]]


In [39]:
# Classification for Label 'Genus'

# best parameters for class 2: 'Genus' =>{'C': 100, 'decision_function_shape': 'ovr', 'gamma': 0.7, 'kernel': 'rbf'}
svm_clf = svm.SVC(kernel='rbf', gamma=0.7, C=100, decision_function_shape='ovr')
svm_clf = svm_clf.fit(X_train,y_train.iloc[:,1])

expected = y_test.iloc[:,1]
predicted = svm_clf.predict(X_test)

hamming_loss_2 = hamming_loss(y_test.iloc[:,1],predicted)
exact_match_2 = accuracy_score(y_test.iloc[:,1],predicted)

print("Hamming loss for Class 2 (Genus):",hamming_loss_2)
print("Exact Match for Class 2 (Genus):",exact_match_2)

print('Classification Report:')
print(metrics.classification_report(expected, predicted))
print('Confusion Matrix:')
print(metrics.confusion_matrix(expected, predicted))

Hamming loss for Class 2 (Genus): 0.01157943492357573
Exact Match for Class 2 (Genus): 0.9884205650764243
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1240
           1       1.00      0.99      1.00       155
           2       0.94      0.97      0.95        96
           3       0.99      0.98      0.99       486
           4       0.99      0.99      0.99        87
           5       0.89      0.86      0.87        28
           6       0.95      0.90      0.92        20
           7       0.98      0.98      0.98        47

    accuracy                           0.99      2159
   macro avg       0.97      0.96      0.96      2159
weighted avg       0.99      0.99      0.99      2159

Confusion Matrix:
[[1235    0    4    0    0    0    0    1]
 [   0  154    1    0    0    0    0    0]
 [   3    0   93    0    0    0    0    0]
 [   3    0    1  478    1    3    0    0]
 [   0    0    0    1   86    

In [41]:
# Classification for Label 'Species'
# best parameters for class 3: 'Species' =>{'C': 1000, 'decision_function_shape': 'ovr', 'kernel': 'rbf'}
svm_clf = svm.SVC(kernel='rbf', C=1000, decision_function_shape='ovr')
svm_clf = svm_clf.fit(X_train,y_train.iloc[:,2])

expected = y_test.iloc[:,2]
predicted = svm_clf.predict(X_test)

hamming_loss_3 = hamming_loss(y_test.iloc[:,2],predicted)
exact_match_3 = accuracy_score(y_test.iloc[:,2],predicted)

print("Hamming loss for label 3 (Species):",hamming_loss_3)
print("Exact Match for label 3 (Species):",exact_match_3)

print('Classification Report:')
print(metrics.classification_report(expected, predicted))
print('Confusion Matrix:')
print(metrics.confusion_matrix(expected, predicted))

Hamming loss for label 3 (Species): 0.012505789717461788
Exact Match for label 3 (Species): 0.9874942102825383
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98       211
           1       1.00      1.00      1.00      1029
           2       1.00      0.99      1.00       155
           3       0.94      0.97      0.95        96
           4       0.96      0.99      0.98       151
           5       0.99      0.98      0.99       335
           6       0.99      0.99      0.99        87
           7       0.89      0.86      0.87        28
           8       0.95      0.90      0.92        20
           9       0.98      0.98      0.98        47

    accuracy                           0.99      2159
   macro avg       0.97      0.96      0.97      2159
weighted avg       0.99      0.99      0.99      2159

Confusion Matrix:
[[ 204    0    0    4    1    1    0    0    0    1]
 [   0 1029    0    0    0    0    0

In [ ]:
#Part iii

from sklearn.svm import LinearSVC


model = svm.LinearSVC(penalty='l1',dual=False)
scores_1 = cross_val_score(model, X=X_train , y=y_train.iloc[:,0] , cv=10 )
scores_2 = cross_val_score(model, X=X_train , y=y_train.iloc[:,1] , cv=10 )
scores_3 = cross_val_score(model, X=X_train , y=y_train.iloc[:,2] , cv=10 )

print('mean of Scores for Class 1:',scores_1.mean())
print('mean of Scores for Class 2:',scores_2.mean())
print('mean of Scores for Class 3:',scores_3.mean())

In [42]:
#iii : L1-penalized SVM

# Finding the best parameters
parameters = {'C' : 10. ** np.arange(-4, 4)}

svm_clf = GridSearchCV(estimator=model,
                     param_grid=parameters,
                     scoring='accuracy',
                     cv=10,
                     n_jobs=-1)

svm_clf.fit(X_train, y_train.iloc[:,0])
print('Best parameter for class 1(Family):',svm_clf.best_params_)
print('Best Score for class 1(Family):',svm_clf.best_score_)

svm_clf.fit(X_train, y_train.iloc[:,1])
print('Best parameter for class 2(Genus):',svm_clf.best_params_)
print('Best Score for class 2(Genus):',svm_clf.best_score_)

svm_clf.fit(X_train, y_train.iloc[:,2])
print('Best parameter for class 3(Species):',svm_clf.best_params_)
print('Best Score for class 3(Species):',svm_clf.best_score_)



Best parameter for class 1(Family): {'C': 100.0}
Best Score for class 1(Family): 0.990864337782827
Best parameter for class 2(Genus): {'C': 100.0}
Best Score for class 2(Genus): 0.9900710814478211
Best parameter for class 3(Species): {'C': 1000.0}
Best Score for class 3(Species): 0.9892762472782355


In [43]:

# best parameters for class 1: Family => {'C': 10}
svm_clf = svm.LinearSVC(C=100 , penalty='l1',dual=False)
svm_clf = svm_clf.fit(X_train,y_train.iloc[:,0])

expected = y_test.iloc[:,0]
predicted = svm_clf.predict(X_test)

hamming_loss_1 = hamming_loss(y_test.iloc[:,0],predicted)
exact_match_1 = accuracy_score(y_test.iloc[:,0],predicted)

print("Hamming loss for Class 1 (Family):",hamming_loss_1)
print("Exact Match for Class 1 (Family):",exact_match_1)

print('Classification Report:')
print(metrics.classification_report(expected, predicted))
print('Confusion Matrix:')
print(metrics.confusion_matrix(expected, predicted))

Hamming loss for Class 1 (Family): 0.06762389995368226
Exact Match for Class 1 (Family): 0.9323761000463178
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        20
           1       0.85      0.92      0.89       155
           2       0.92      0.89      0.91       657
           3       0.95      0.97      0.96      1327

    accuracy                           0.93      2159
   macro avg       0.68      0.70      0.69      2159
weighted avg       0.92      0.93      0.93      2159

Confusion Matrix:
[[   0    0   18    2]
 [   0  143    8    4]
 [   1    5  588   63]
 [   0   20   25 1282]]


In [ ]:
svm_clf = svm.LinearSVC(C=100 , penalty='l1',dual=False)
svm_clf = svm_clf.fit(X_train,y_train.iloc[:,1])

expected = y_test.iloc[:,0]
predicted = svm_clf.predict(X_test)

hamming_loss_2 = hamming_loss(y_test.iloc[:,1],predicted)
exact_match_2 = accuracy_score(y_test.iloc[:,1],predicted)

print("Hamming loss for Class 2 (Genus):",hamming_loss_2)
print("Exact Match for Class 2 (Genus):",exact_match_2)

print('Classification Report:')
print(metrics.classification_report(expected, predicted))
print('Confusion Matrix:')
print(metrics.confusion_matrix(expected, predicted))

In [ ]:
svm_clf = svm.LinearSVC(C=1000 , penalty='l1',dual=False)
svm_clf = svm_clf.fit(X_train,y_train.iloc[:,2])

expected = y_test.iloc[:,0]
predicted = svm_clf.predict(X_test)

hamming_loss_3 = hamming_loss(y_test.iloc[:,2],predicted)
exact_match_3 = accuracy_score(y_test.iloc[:,2],predicted)

print("Hamming loss for label 3 (Species):",hamming_loss_3)
print("Exact Match for label 3 (Species):",exact_match_3)

print('Classification Report:')
print(metrics.classification_report(expected, predicted))
print('Confusion Matrix:')
print(metrics.confusion_matrix(expected, predicted))

In [ ]:
# y_pred= np.concatenate((predicted_1.reshape(-1,1),predicted_2.reshape(-1,1),predicted_3.reshape(-1,1)),axis=1)
# y_real= np.concatenate((np.array(y_test.iloc[:,0]).reshape(-1,1),np.array(y_test.iloc[:,1]).reshape(-1,1),np.array(y_test.iloc[:,2]).reshape(-1,1)),axis=1)

# HM= np.sum(np.not_equal(y_real, y_pred))/float(y_real.size)
# print("Overall Hamming loss is",HM)

# val=0
# l=len(y_pred)
# for i in range(0,l):
#     if(predicted_1[i]==y_test.iloc[i,0] and predicted_2[i]==y_test.iloc[i,1] and predicted_3[i]==y_test.iloc[i,2]):
#         val=val+1
# val=val/l
# print("Exact Match score is",val)

In [49]:
#iv. SMOTE

from imblearn.over_sampling import SMOTE 

seed = 100
k = 5
# X_res, y_res = make_imbalance(X1, Y1, sampling_strategy={0: 400, 1: 10, 2: 50 , 3: 100 }, random_state=seed)

sm = SMOTE(sampling_strategy='auto', k_neighbors=k, random_state=seed)
Xt1,yt1 = sm.fit_resample(X_train,y_train.iloc[:,0])
Xt2,yt2 = sm.fit_resample(X_train,y_train.iloc[:,1])
Xt3,yt3 = sm.fit_resample(X_train,y_train.iloc[:,2])


# model = svm.LinearSVC(penalty='l1',dual=False)
# scores_1 = cross_val_score(model, Xt1, yt1, cv=10 )
# scores_2 = cross_val_score(model, Xt2, yt2, cv=10 )
# scores_3 = cross_val_score(model, Xt3, yt3, cv=10 )

# print('mean of Scores for Class 1:',scores_1.mean())
# print('mean of Scores for Class 2:',scores_2.mean())
# print('mean of Scores for Class 3:',scores_3.mean())

In [53]:
svm_clf = svm.LinearSVC(penalty='l1',dual=False)
svm_clf = svm_clf.fit(Xt1,yt1)

expected = y_test.iloc[:,0]
predicted = svm_clf.predict(X_test)

print('Classification Report:')
print(metrics.classification_report(expected, predicted))
print('Confusion Matrix:')
print(metrics.confusion_matrix(expected, predicted))

Classification Report:
              precision    recall  f1-score   support

           0       0.29      0.90      0.43        20
           1       0.70      0.98      0.82       155
           2       0.94      0.86      0.90       657
           3       0.96      0.93      0.94      1327

    accuracy                           0.91      2159
   macro avg       0.72      0.92      0.77      2159
weighted avg       0.93      0.91      0.92      2159

Confusion Matrix:
[[  18    1    0    1]
 [   0  152    2    1]
 [  24   22  567   44]
 [  21   42   35 1229]]


In [54]:
svm_clf = svm.LinearSVC()
svm_clf = svm_clf.fit(Xt2,yt2)

expected = y_test.iloc[:,1]
predicted = svm_clf.predict(X_test)

print('Classification Report:')
print(metrics.classification_report(expected, predicted))
print('Confusion Matrix:')
print(metrics.confusion_matrix(expected, predicted))

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.90      0.93      1240
           1       0.78      0.96      0.86       155
           2       0.62      0.80      0.70        96
           3       0.98      0.90      0.94       486
           4       0.93      0.94      0.94        87
           5       0.34      0.86      0.49        28
           6       0.41      0.90      0.56        20
           7       0.90      0.98      0.94        47

    accuracy                           0.90      2159
   macro avg       0.74      0.91      0.80      2159
weighted avg       0.93      0.90      0.91      2159

Confusion Matrix:
[[1113   40   40    4    0   30   11    2]
 [   1  149    4    1    0    0    0    0]
 [  10    2   77    2    0    0    5    0]
 [  15    0    1  438    6   15    9    2]
 [   0    0    2    2   82    1    0    0]
 [   1    0    0    2    0   24    0    1]
 [   2    0    0    0    0    0   18    0]
 [   0 

In [55]:
svm_clf = svm.LinearSVC()
svm_clf = svm_clf.fit(Xt3,yt3)

expected = y_test.iloc[:,2]
predicted = svm_clf.predict(X_test)

print('Classification Report:')
print(metrics.classification_report(expected, predicted))
print('Confusion Matrix:')
print(metrics.confusion_matrix(expected, predicted))

Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.91      0.92       211
           1       0.99      1.00      0.99      1029
           2       0.91      0.95      0.93       155
           3       0.89      0.86      0.88        96
           4       0.93      0.93      0.93       151
           5       0.96      0.91      0.93       335
           6       0.95      0.94      0.95        87
           7       0.73      0.86      0.79        28
           8       0.58      0.90      0.71        20
           9       0.92      0.98      0.95        47

    accuracy                           0.95      2159
   macro avg       0.88      0.92      0.90      2159
weighted avg       0.96      0.95      0.95      2159

Confusion Matrix:
[[ 191    0   11    5    0    1    0    1    1    1]
 [   0 1024    1    0    0    0    0    0    1    3]
 [   3    0  147    2    0    2    0    0    1    0]
 [   6    0    2   83    0    2    0    0